In [20]:
pip install pandas selenium webdriver-manager beautifulsoup4


In [13]:
from datetime import datetime, timedelta

def create_url(people, country, city, date_in, date_out, rooms, rows=100):
    url = f"https://www.booking.com/searchresults.en-gb.html?selected_currency=USD&checkin_month={date_in.month}" \
          f"&checkin_monthday={date_in.day}&checkin_year={date_in.year}&checkout_month={date_out.month}" \
          f"&checkout_monthday={date_out.day}&checkout_year={date_out.year}&group_adults={people}" \
          f"&group_children=0&order=price&ss={city}%2C%20{country}" \
          f"&no_rooms={rooms}"
    return url

# Define parameters
people = 2
country = "United States"
city = "New York"
rooms = 1
URLS=[]

# Iterate over all combinations of TTT and LOS
for ttt in range(1, 3):
    for los in range(1, 3):
        # Define date snapshot (date you do the site sampling)
        date_snapshot = datetime.now()
        
        # Calculate check-in and check-out dates based on TTT and LOS
        date_in = date_snapshot + timedelta(days=ttt)
        date_out = date_in + timedelta(days=los)
        
        # Create URL using the create_url function
        url = create_url(people, country, city, date_in, date_out, rooms)
        URLS.append(url)
        print(url)

https://www.booking.com/searchresults.en-gb.html?selected_currency=USD&checkin_month=5&checkin_monthday=22&checkin_year=2024&checkout_month=5&checkout_monthday=23&checkout_year=2024&group_adults=2&group_children=0&order=price&ss=New York%2C%20United States&no_rooms=1
https://www.booking.com/searchresults.en-gb.html?selected_currency=USD&checkin_month=5&checkin_monthday=22&checkin_year=2024&checkout_month=5&checkout_monthday=24&checkout_year=2024&group_adults=2&group_children=0&order=price&ss=New York%2C%20United States&no_rooms=1
https://www.booking.com/searchresults.en-gb.html?selected_currency=USD&checkin_month=5&checkin_monthday=23&checkin_year=2024&checkout_month=5&checkout_monthday=24&checkout_year=2024&group_adults=2&group_children=0&order=price&ss=New York%2C%20United States&no_rooms=1
https://www.booking.com/searchresults.en-gb.html?selected_currency=USD&checkin_month=5&checkin_monthday=23&checkin_year=2024&checkout_month=5&checkout_monthday=25&checkout_year=2024&group_adults=2

In [14]:
len (URLS)

4

In [12]:
import pandas as pd

# Temporarily set display options
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(global_hotel_data)


                                                  name  \
0                                               HUDSON   
1                                            Hotel 309   
2                                 Night Hotel Broadway   
3                             31 Street Broadway Hotel   
4                                    Lamartine Chelsea   
5                                       Hotel Moca NYC   
6                       The Nolita Express 1927 Hostel   
7             THE CENTRAL PARK EAST by THE MUSEUM MILE   
8                                          Doxie Hotel   
9                                              SoHo 54   
10                                Hotel 27 by LuxUrban   
11   Fairfield Inn by Marriott New York Manhattan/F...   
12                                         Radio Hotel   
13   Courtyard by Marriott New York World Trade Cen...   
14                                      The FIDI Hotel   
15                                    Pod Times Square   
16            

In [30]:
# Define the file path
csv_file_path = r"C:\Users\ronro\crowling_crowling\hotels_data.csv"

# Export the DataFrame to a CSV file
all_hotels_df.to_csv(csv_file_path, index=False)

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define the column names
columns = [
    'name', 'location', 'score', 'reviews', 'room_type', 'bed_type', 'price', 
    'availability', 'url', 'star_rating', 'location_rating', 'Snapshot',
    'Free_cancellation', 'No_prepayment', 'Limited_rooms', 'TTT'
]

# Reset the DataFrame with only column names
all_hotels_df = pd.DataFrame(columns=columns)

def create_url(people, country, city, date_in, date_out, rooms):
    url = f"https://www.booking.com/searchresults.en-gb.html?selected_currency=USD&checkin_month={date_in.month}" \
          f"&checkin_monthday={date_in.day}&checkin_year={date_in.year}&checkout_month={date_out.month}" \
          f"&checkout_monthday={date_out.day}&checkout_year={date_out.year}&group_adults={people}" \
          f"&group_children=0&order=price&ss={city}%2C%20{country}" \
          f"&no_rooms={rooms}"
    return url

def fetch_hotel_data(url, ttt):
    global all_hotels_df
    
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("start-maximized")
    chrome_options.add_argument("disable-infobars")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--headless")  # Run headless if you don't need a browser UI
    chrome_options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
    
    # Set up Selenium WebDriver for Chrome
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Create an empty list to store hotel data
    hotel_data = []

    # Today's date
    snapshot_date = datetime.today().strftime('%Y-%m-%d')

    try:
        # Navigate to the URL
        driver.get(url)

        
        def scroll_down():
            last_height = 0
            while True:
                try:
                    # Check for the pop-up and close it if it appears
                    close_button = driver.find_element(By.XPATH, "//button[@aria-label='Dismiss sign in information.']")
                    if close_button.is_displayed():
                        close_button.click()
                        time.sleep(2)  # Wait for the pop-up to be dismissed
                except Exception as e:
                    # Pop-up did not appear, continue scrolling
                    pass
        
                # Scroll down to the bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                # Wait for some time to load new content
                time.sleep(2)
                # Calculate new height of the page after scrolling
                new_height = driver.execute_script("return document.body.scrollHeight")
                # Break the loop if height remains the same (i.e., no more content to load)
                if new_height == last_height:
                    break
                last_height = new_height


        # Loop to scroll and click "Load more results" until no more results are loaded or 180 hotels are collected
        while len(hotel_data) < 180:
            scroll_down()  # Scroll down the page
            time.sleep(2)  # Wait for the page to stabilize
            try:
                # Scroll to the button
                load_more_button = driver.find_element(By.XPATH, "//button[contains(@class, 'a83ed08757') and contains(@class, 'c21c56c305') and contains(@class, 'bf0537ecb5') and contains(@class, 'f671049264') and contains(@class, 'deab83296e') and contains(@class, 'af7297d90d')]")
                driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
                time.sleep(2)  # Wait for the button to be in the view
                # Wait for the button to be clickable
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'a83ed08757') and contains(@class, 'c21c56c305') and contains(@class, 'bf0537ecb5') and contains(@class, 'f671049264') and contains(@class, 'deab83296e') and contains(@class, 'af7297d90d')]")))
                load_more_button.click()
                time.sleep(5)  # Wait for new results to load
            except Exception as e:
                print("No more results to load or error:", e)
                break

        # Get the page source after dynamic content is loaded
        html = driver.page_source

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find all hotel cards
        hotel_tabs = soup.find_all('div', class_="c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 c6710787a4")

        # Extract hotel information and append to list
        for tab in hotel_tabs:
            if len(hotel_data) >= 180:
                break
            try:
                name = tab.find('div', class_='f6431b446c a15b38c233').text.strip()
            except AttributeError:
                name = "N/A"
            try:
                location = tab.find('span', class_='aee5343fdb def9bc142a').text.strip()
            except AttributeError:
                location = "N/A"
            try:
                score = tab.find('div', class_='a3b8729ab1 d86cee9b25').text.strip()
            except AttributeError:
                score = "N/A"
            try:
                reviews = tab.find('div', class_='abf093bdfe f45d8e4c32 d935416c47').text.strip()
            except AttributeError:
                reviews = "N/A"
            try:
                room_type = tab.find('h4', class_='abf093bdfe e8f7c070a7').text.strip()
            except AttributeError:
                room_type = "N/A"
            try:
                bed_type = tab.find('div', class_='abf093bdfe').text.strip()
            except AttributeError:
                bed_type = "N/A"
            try:
                price = tab.find('span', class_='f6431b446c fbfd7c1165 e84eb96b1f').text.strip()
            except AttributeError:
                price = "N/A"
            try:
                availability = tab.find('div', class_='abf093bdfe f45d8e4c32').text.strip()
            except AttributeError:
                availability = "N/A"
            try:
                url = tab.find('a', class_='a78ca197d0')['href']
            except TypeError:
                url = "N/A"
            try:
                star_rating = tab.find('div', {'aria-label': True})['aria-label']
            except TypeError:
                star_rating = "N/A"
            try:
                location_rating_link = tab.find('a', class_='a83ed08757 f88a5204c2 b98133fb50')
                location_rating = location_rating_link['aria-label'].split()[-1] if location_rating_link else "N/A"
            except TypeError:
                location_rating = "N/A"
            try:
                free_cancellation = "Yes" if tab.find('strong', text="Free cancellation") else "No"
            except AttributeError:
                free_cancellation = "No"
            try:
                no_prepayment = "Yes" if tab.find('strong', text="No prepayment needed") else "No"
            except AttributeError:
                no_prepayment = "No"
            try:
                limited_rooms = tab.find('div', class_='d17181842f').text.strip() if tab.find('div', class_='d17181842f') else "No"
            except AttributeError:
                limited_rooms = "No"

            # Append the information as a dictionary to the list
            hotel_data.append({
                'name': name,
                'location': location,
                'score': score,
                'reviews': reviews,
                'room_type': room_type,
                'bed_type': bed_type,
                'price': price,
                'availability': availability,
                'url': url,
                'star_rating': star_rating,
                'location_rating': location_rating,
                'Snapshot': snapshot_date,
                'Free_cancellation': free_cancellation,
                'No_prepayment': no_prepayment,
                'Limited_rooms': limited_rooms,
                'TTT': ttt
            })

    except Exception as e:
        print("An error occurred:", e)

    finally:
        # Close the WebDriver
        driver.quit()

    # Convert the collected data to a DataFrame
    new_hotels_df = pd.DataFrame(hotel_data)

    # Append new data to the existing DataFrame
    all_hotels_df = pd.concat([all_hotels_df, new_hotels_df], ignore_index=True)

# Define parameters
people = 2
country = "United States"
city = "New York"
rooms = 1
URLS = []

# Iterate over all combinations of TTT and LOS
for ttt in range(1, 31):
    for los in range(1, 6):
        # Define date snapshot (date you do the site sampling)
        date_snapshot = datetime.now()
        
        # Calculate check-in and check-out dates based on TTT and LOS
        date_in = date_snapshot + timedelta(days=ttt)
        date_out = date_in + timedelta(days=los)
        
        # Create URL using the create_url function
        url = create_url(people, country, city, date_in, date_out, rooms)
        URLS.append((url, ttt))

# Define a counter variable
counter = 0

# Fetch hotel data for each URL
for url, ttt in URLS:
    # Increment the counter
    counter += 1
    
    # Print the current count
    print(f"Function activated {counter} times")
    
    # Call the function
    fetch_hotel_data(url, ttt)


# Define the file path
csv_file_path = r"C:\Users\ronro\crowling_crowling\hotels_data.csv"

# Export the DataFrame to a CSV file
all_hotels_df.to_csv(csv_file_path, index=False)
# Print the final DataFrame
print(all_hotels_df)


Function activated 1 times
No more results to load or error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(@class, 'a83ed08757') and contains(@class, 'c21c56c305') and contains(@class, 'bf0537ecb5') and contains(@class, 'f671049264') and contains(@class, 'deab83296e') and contains(@class, 'af7297d90d')]"}
  (Session info: chrome-headless-shell=125.0.6422.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0064B793+45827]
	(No symbol) [0x005DDB74]
	(No symbol) [0x004D150F]
	(No symbol) [0x005120BC]
	(No symbol) [0x0051216B]
	(No symbol) [0x0054E0F2]
	(No symbol) [0x00532E44]
	(No symbol) [0x0054C034]
	(No symbol) [0x00532B96]
	(No symbol) [0x00506998]
	(No symbol) [0x0050751D]
	GetHandleVerifier [0x009043C3+2899763]
	GetHandleVerifier [0x009577ED+3240797]
	GetHandleVerifier [0x006D1264+593364]
	GetHandl

C:\Users\ronro\AppData\Local\Temp\ipykernel_13512\1758050594.py:158: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  free_cancellation = "Yes" if tab.find('strong', text="Free cancellation") else "No"
C:\Users\ronro\AppData\Local\Temp\ipykernel_13512\1758050594.py:162: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  no_prepayment = "Yes" if tab.find('strong', text="No prepayment needed") else "No"


Function activated 2 times
No more results to load or error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(@class, 'a83ed08757') and contains(@class, 'c21c56c305') and contains(@class, 'bf0537ecb5') and contains(@class, 'f671049264') and contains(@class, 'deab83296e') and contains(@class, 'af7297d90d')]"}
  (Session info: chrome-headless-shell=125.0.6422.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0064B793+45827]
	(No symbol) [0x005DDB74]
	(No symbol) [0x004D150F]
	(No symbol) [0x005120BC]
	(No symbol) [0x0051216B]
	(No symbol) [0x0054E0F2]
	(No symbol) [0x00532E44]
	(No symbol) [0x0054C034]
	(No symbol) [0x00532B96]
	(No symbol) [0x00506998]
	(No symbol) [0x0050751D]
	GetHandleVerifier [0x009043C3+2899763]
	GetHandleVerifier [0x009577ED+3240797]
	GetHandleVerifier [0x006D1264+593364]
	GetHandl

In [2]:
all_hotels_df.shape


(26845, 16)